In [4]:
pip install --upgrade tensorflow

     |████████████████████████████████| 458.4 MB 69 kB/s  eta 0:00:01
     |████████████████████████████████| 4.4 MB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 14.9 MB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.9 MB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 154 kB 5.

In [11]:
pip install -U prettytable

Note: you may need to restart the kernel to use updated packages.


In [12]:
# Processing imports

import pandas as pd
import numpy as np
import glob
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact
from pathlib import Path
import joblib
from sklearn.pipeline import make_pipeline
from prettytable import PrettyTable

%matplotlib inline

In [6]:
# ML imports

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler  


# Data gathering and pre-processing

In [340]:
# Path to data directory
path = r'/home/connor/Desktop/Pastebles/data'

# Holder list
li = []

# Regex changes the target class creation
# 'CORRECT|WRONG|[a-zA-Z]+(?=_[0-9]+_data)|(?<=IR_[0-9]_)[a-zA-Z]+'

# Will grab any subfolders from path and their csv files

for filename in Path(path).rglob('*.csv'):
    # Reads individual csv files
    # print(filename)
    # print("\n")
    df = pd.read_csv(filename, index_col=None, header=0)
    # Matches specifics from the filename using regex (subject to change depending on filenaming convention)   
    match  = re.findall('[a-zA-Z]+(?=_[0-9]+_data)', str(filename))
    y = (''.join(match))
    # Adds target column for classification
    df['y'] = y
    df['filename'] = filename
    # Appends the dataframe to the list
    li.append(df)

# Concats all data into one dataframe for training/testing
frame = pd.concat(li, axis=0, ignore_index=True)

# Target column is y
y_string = frame['y']
actions = ['BicepCurls', 'Posture', 'SideLunges', 'Sitting', 'Squats', 'Standing']

# Changes target from string to numeric
le = LabelEncoder().fit(actions)
y = le.transform(y_string.ravel())

print(le.classes_)
print(le.transform(le.classes_))

# Sets the X data
filelist = frame['filename']
X = frame.drop(['y','arrival_time', 'filename'],axis=1).to_numpy()

# Splits data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20) # random_state=42

['BicepCurls' 'Posture' 'SideLunges' 'Sitting' 'Squats' 'Standing']
[0 1 2 3 4 5]


In [296]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)

In [341]:
frame['ir_dist'].min()

4

In [342]:
pd.DataFrame(X_test)[6].min()

4.0

In [192]:
all_classes = le.classes_
print("All Classes:\n", *all_classes, sep = "\n")

All Classes:

BicepCurls
Posture
SideLunges
Sitting
Squats
Standing


# Model creation and training

In [321]:
# Neural network creation
# input layer -> hidden layer (32 nodes) -> hidden layer (16 nodes) -> output layer (amount of classes)

model = tf.keras.Sequential()
# model.add(tf.keras.Input(shape=(7,)))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(all_classes.size)), # activation='softmax'))

opt = tf.keras.optimizers.Adam()

model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# scaler = StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test) 

model.fit(X_train, y_train, epochs=50)
model.summary()

Epoch 1/50
3138/3138 [==============================] - 1s 396us/step - loss: 1.4292 - accuracy: 0.4605
Epoch 2/50
3138/3138 [==============================] - 1s 398us/step - loss: 1.1276 - accuracy: 0.5625
Epoch 3/50
3138/3138 [==============================] - 1s 393us/step - loss: 1.0334 - accuracy: 0.5995
Epoch 4/50
3138/3138 [==============================] - 1s 388us/step - loss: 0.9610 - accuracy: 0.6206
Epoch 5/50
3138/3138 [==============================] - 1s 405us/step - loss: 0.9217 - accuracy: 0.6313
Epoch 6/50
3138/3138 [==============================] - 1s 436us/step - loss: 0.8879 - accuracy: 0.64500s - los
Epoch 7/50
3138/3138 [==============================] - 1s 465us/step - loss: 0.8629 - accuracy: 0.6534 - ETA: 0s - loss: 0.8630 - accuracy: 0.
Epoch 8/50
3138/3138 [==============================] - 1s 432us/step - loss: 0.8398 - accuracy: 0.6600
Epoch 9/50
3138/3138 [==============================] - 1s 428us/step - loss: 0.8212 - accuracy: 0.66340s - loss: 0.8228

# Accuracy scoring

In [322]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

785/785 - 0s - loss: 0.6255 - accuracy: 0.7452

Test accuracy: 0.745179295539856


# Individual checking

In [323]:
# If output layer needs to be converted to probabilities
# probability_model = tf.keras.Sequential([model, 
#                                          tf.keras.layers.Softmax()])
# predictions = probability_model.predict(X_test)

# If softmax activation is used for output layer
predictions = model.predict(X_test) 

In [334]:
# Single prediction from manually chosen data
singleAction = (np.expand_dims([-0.157470703125,0.8681640625,-0.41455078125,1.77001953125,-4.8828125,-2.86865234375, 7],0))
singleAction

array([[-0.1574707 ,  0.86816406, -0.41455078,  1.77001953, -4.8828125 ,
        -2.86865234,  7.        ]])

In [339]:
singlePrediction = model.predict(singleAction)
print("Predicted:", le.inverse_transform(np.argmax(singlePrediction).ravel()))

pretty_table = PrettyTable()
pretty_table.add_column("Action",all_classes)
pretty_table.add_column("Probability", singlePrediction[0]*100)
print(pretty_table)

Predicted: ['SideLunges']
+------------+-------------+
|   Action   | Probability |
+------------+-------------+
| BicepCurls |   433.5578  |
|  Posture   |  -912.7257  |
| SideLunges |  822.50574  |
|  Sitting   |  -1467.5808 |
|   Squats   |   820.1295  |
|  Standing  |  -1656.9595 |
+------------+-------------+


In [336]:
# All predictions from X_test
index = 0
print("Predicted:", le.inverse_transform(np.argmax(predictions[index]).ravel()), "Actual:", le.inverse_transform(y_test[index].ravel()))

pretty_table = PrettyTable()
pretty_table.add_column("Action",all_classes)
pretty_table.add_column("Probability", predictions[index]*100)
print(pretty_table)

Predicted: ['Posture'] Actual: ['Posture']
+------------+-------------+
|   Action   | Probability |
+------------+-------------+
| BicepCurls |   660.5464  |
|  Posture   |  1030.9507  |
| SideLunges |  833.27216  |
|  Sitting   |  -1133.3273 |
|   Squats   |  728.61365  |
|  Standing  |  -1620.3573 |
+------------+-------------+


In [331]:
# Saving the tensorflow model
model.save('actionModel')

INFO:tensorflow:Assets written to: actionModel/assets


In [332]:
# Converting the tfmodel to a tflite model
converter = tf.lite.TFLiteConverter.from_saved_model('actionModel') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('converted_model_tflite', 'wb') as f:
    f.write(tflite_model)

In [177]:
# # Path to data directory
# path = r'/home/connor/Desktop/Pastebles/data'

# # Holder list
# li = []

# # Regex changes the target class creation
# # 'CORRECT|WRONG|[a-zA-Z]+(?=_[0-9]+_data)|(?<=IR_[0-9]_)[a-zA-Z]+'

# # Will grab any subfolders from path and their csv files

# for filename in Path(path).rglob('*.csv'):
#     # Reads individual csv files
#     df = pd.read_csv(filename, index_col=None, header=0)
#     # Matches specifics from the filename using regex (subject to change depending on filenaming convention)   
#     match  = re.findall('[a-zA-Z]+(?=_[0-9]+_data)', str(filename))
#     y = (''.join(match))
#     # Adds target column for classification
#     df['y'] = y
#     # Appends the dataframe to the list
#     li.append(df)

# # Concats all data into one dataframe for training/testing
# frame = pd.concat(li, axis=0, ignore_index=True)

# # Target column is y
# y_string = frame['y']
# actions = ['BicepCurls', 'Posture', 'SideLunges', 'Sitting', 'Squats', 'Standing']

# # Changes target from string to numeric
# le = LabelEncoder().fit(actions)
# y = le.transform(y_string.ravel())

# print(le.classes_)
# print(le.transform(le.classes_))

# # Sets the X data
# X = frame.drop(['y','arrival_time'],axis=1).to_numpy()

# # Splits data into training and testing
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20) # random_state=42

['BicepCurls' 'Posture' 'SideLunges' 'Sitting' 'Squats' 'Standing']
[0 1 2 3 4 5]
